In [1]:
import numpy as np
import threading
from threading import Thread
import pandas as pd
import re
import nltk
import spacy
import string
import queue
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from sklearn.decomposition import TruncatedSVD
from io import StringIO
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import time
from collections import Counter
from subprocess import call

In [19]:
df = pd.read_csv('./1 Billion Citation Dataset/1 Billion Citation Dataset, v1 (104).csv')
df = pd.DataFrame(df)


In [20]:
df1 = df[:250000]
df2 = df[250000:500000]
df3 = df[500000:750000]
df4 = df[750000:1000000]
df5 = df[1000000:1250000]
df6 = df[1250000:1500000]
df7 = df[1500000:1750000]
df8 = df[1750000:2000000]
df9 = df[2250000:2500000]



In [21]:
def remove_urls(text):
  url_pattern = re.compile(r"https?://\S+|www\.\S+")
  return url_pattern.sub(r"", text)

def remove_html(text):
  html_pattern = re.compile("<.*?>")
  return html_pattern.sub(r"", text)

In [23]:
PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","" , PUNCT))

In [24]:

nltk.download('stopwords')
",".join(stopwords.words("english"))
STOPW = set(stopwords.words("english"),)
def remove_stopwords(text):
  return" ".join([word for word in str(text).split() if word not in STOPW])        

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def preprocess(df):

    df_preprocess = df.loc[:,'citationStringAnnotated']
    df_preprocess = df_preprocess.astype(str)
    df_preprocess = df_preprocess.str.lower()
    df_preprocess = df_preprocess.apply(lambda text: remove_urls(text))
    df_preprocess = df_preprocess.apply(lambda text: remove_html(text))
    df_preprocess= df_preprocess.apply(lambda text: remove_punctuation(text))
    df_preprocess = df_preprocess.apply(lambda text: remove_stopwords(text))
    df_preprocess = df_preprocess.str.replace('\d+', '',regex=True)

    return df_preprocess

In [26]:
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [27]:

dfa = []
thread1 = ThreadWithReturnValue(target = preprocess, args=(df1,))
thread2 = ThreadWithReturnValue(target = preprocess, args=(df2,))
thread3 = ThreadWithReturnValue(target = preprocess, args=(df3,))
thread4 = ThreadWithReturnValue(target = preprocess, args=(df4,))
thread5 = ThreadWithReturnValue(target = preprocess, args=(df5,))
thread6 = ThreadWithReturnValue(target = preprocess, args=(df6,))
thread7 = ThreadWithReturnValue(target = preprocess, args=(df7,))
thread8 = ThreadWithReturnValue(target = preprocess, args=(df8,))
thread9 = ThreadWithReturnValue(target = preprocess, args=(df9,))

thread1.start()
df1 = thread1.join()

thread2.start()
df2 = thread2.join()

thread3.start()
df3 = thread3.join()

thread4.start()
df4 = thread4.join()

thread5.start()
df5 = thread5.join()

thread6.start()
df6 = thread6.join()

thread7.start()
df7 = thread7.join()

thread8.start()
df8 = thread8.join()

thread9.start()
df9 = thread9.join()

<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>


In [10]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9]
result = pd.concat(frames)
display(result)

0          hashmi mg kisat mt rehman ka zafar et al  “ris...
1          sakama makoto kanematsu nobuyuki  “an evaluati...
2          stephenson andrew argent stephen p riisjohanne...
3          mujdeci b onder allusoglu i̇riz et al  “saccul...
4          geng wangchang zhou lunwei duan libing gu junw...
                                 ...                        
2499995    seongdeok ahn yoonho song sungyool choi et al ...
2499996    peters l  h smit v pechota rules national arbi...
2499997    xu zhu j gao q zhu b  sailing routes designing...
2499998    abad ninet monserrat molas j  principio de dis...
2499999    kumar n beer jd vanthienen j moens mf comprehe...
Name: citationStringAnnotated, Length: 2250000, dtype: object

In [11]:
def build_lsa(dataset, dim, queue ):
        tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
        svd = TruncatedSVD(n_components=dim)
    
        transformed_x_train = tfidf_vec.fit_transform(dataset)
    
        print('TF-IDF output shape:', transformed_x_train.shape)
    
        data_svd = svd.fit_transform(transformed_x_train)
    
        print('LSA output shape:', data_svd.shape)
    
        explained_variance = svd.explained_variance_ratio_.sum()
        print("Sum of explained variance ratio: %d%%" % (int(explained_variance * 100)))

        queue.put((tfidf_vec, transformed_x_train,svd, data_svd))
         

In [12]:
queue1 = queue.Queue()
thread_lsa1 = threading.Thread(target = build_lsa, args = (result,50, queue1,)) 
thread_lsa1.start()
thread_lsa1.join()

TF-IDF output shape: (2250000, 71348)
LSA output shape: (2250000, 50)
Sum of explained variance ratio: 7%


In [13]:
tfidf_vec1, transformed_x_train1,svd1, data_svd = queue1.get()

In [15]:
l=data_svd[0]

for i,topic in enumerate(l):
  print("Topic ",i+1," : ",topic*100)

Topic  1  :  15.440115030714486
Topic  2  :  0.10889562998972616
Topic  3  :  -4.362621304722439
Topic  4  :  -4.0515012078383625
Topic  5  :  0.6775582634597357
Topic  6  :  7.1233395162210185
Topic  7  :  11.744918997747513
Topic  8  :  -0.7749267582103175
Topic  9  :  0.8671009030320078
Topic  10  :  4.952853646050313
Topic  11  :  -4.145181778552065
Topic  12  :  -5.389313206164668
Topic  13  :  -1.645524312496752
Topic  14  :  -2.940704998743056
Topic  15  :  4.481165422374671
Topic  16  :  -6.295411063209862
Topic  17  :  -1.2026443988050999
Topic  18  :  -7.040090130950789
Topic  19  :  -1.6527907920279428
Topic  20  :  -0.38085495901005767
Topic  21  :  2.6465464442773508
Topic  22  :  -0.47348719507591364
Topic  23  :  3.88992641255365
Topic  24  :  1.0072055066323666
Topic  25  :  -1.5044403310025478
Topic  26  :  0.8146741724048103
Topic  27  :  3.0708128348905626
Topic  28  :  -0.6914834983931663
Topic  29  :  2.2173366228566964
Topic  30  :  -2.9337234785374458
Topic  31  

In [16]:
vocab = tfidf_vec1.get_feature_names()
list1 = []
list2 = []
for i, comp in enumerate(svd1.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i+1)+": ")
    list1.insert(i,np.array(sorted_words)[:,1])
    list2.append(" ".join(np.array(sorted_words)[:,0]))
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

C:\Users\Thon\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 1: 
journal available pp online doi american international internet et anon 

Topic 2: 
ieee conference international th journal systems proceedings system wang based 

Topic 3: 
de ieee conference international press springer la university pp th 

Topic 4: 
press university oxford pp cambridge ieee online springer conference anon 

Topic 5: 
springer berlin heidelberg pp der nature available und online die 

Topic 6: 
available online anon internet reviews choice ieee book supsup ppp 

Topic 7: 
et al elsevier bv wang research internet zhang li cancer 

Topic 8: 
der und die für des internet von zeitschrift verlag available 

Topic 9: 
society american et available al internet engineers mechanical springer berlin 

Topic 10: 
reviews online et choice al der pp american vol lt 

Topic 11: 
doi review society american reviews internet news nature choice book 

Topic 12: 
elsevier society pp bv vol engineers mechanical chemical physics materials 

Topic 13: 
american elsevier medic

In [17]:
listf = pd.DataFrame(list1)
listf

,0,1,2,3,4,5,6,7,8,9
0,0.37101499710620073,0.32243286067843774,0.253800194424889,0.2094121979690619,0.20520368953738552,0.16275584210811628,0.15462824010335677,0.15078387933715623,0.13616886207223283,0.13161098749235686
1,0.4463012600079569,0.2930038736881213,0.2628040797627901,0.12264844093115956,0.1220156089420153,0.10946307409411671,0.10117759069854909,0.06766272430871897,0.0627538154625826,0.05931326315890816
2,0.3662589614998049,0.33912057872424356,0.2282207870863031,0.14138685726611422,0.14044044577170295,0.13812106862906356,0.12991624079594785,0.11507782523352686,0.10270790533507909,0.10026128719613826
3,0.3763604114955882,0.3605004682268591,0.23381355764345646,0.11883544115752495,0.10242330784332064,0.06795592400910888,0.06577984026256013,0.059404700876836185,0.05684756880397853,0.0554343732992707
4,0.46027369164063353,0.32121071310194754,0.32083572249461606,0.1642903838687213,0.1505626975485221,0.13852568385673755,0.10524908933967635,0.085046630314575,0.07286585779991664,0.0655618828283038
5,0.505050783079541,0.4635681103750682,0.27385009295152407,0.20824451570887673,0.19411626586587294,0.17771237180093916,0.06564815596894495,0.04296792191743278,0.03214292456190364,0.026701181410564625
6,0.44389642048327765,0.43133605835234556,0.15596927588125226,0.1291942915463853,0.09932030140267605,0.0934661665784576,0.08261093638927908,0.07949839243860862,0.07606480937468992,0.06877639320534379
7,0.5269578129796373,0.34621529299206205,0.2809725498671616,0.19339575133365156,0.15119084148414155,0.1405876798512024,0.12476359325925153,0.12185870556160068,0.12032747094889547,0.11596842037662806
8,0.4344111342159341,0.2737172636575811,0.20546344768541402,0.19940234868310402,0.1938295225361981,0.17041140719107356,0.14068008176900876,0.1272207941323347,0.1253463208278246,0.11087456937374693
9,0.31083997047807227,0.298159513171216,0.2611734210523694,0.2556010333554676,0.2533648670440502,0.14445794007009408,0.1418979495894352,0.11665836941373958,0.11508344933886173,0.10988736199556504


In [18]:
list_name = list(list2)

simila = {}

for index in range(listf.shape[0]):

    similarities_1 = linear_kernel(list1[index].reshape(1, -1) ,list1).flatten()
    related_words = (-similarities_1).argsort()[:10]

    simila.update({list_name[index]:[list_name[i] for i in related_words]})

df_results = pd.DataFrame(simila)
df_results.reset_index(inplace=True)
df_results = df_results.T
df_results

,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
journal available pp online doi american international internet et anon,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,american elsevier medical association bv revie...,journal available pp online doi american inter...,elsevier society pp bv vol engineers mechanica...,cheminform abstract nono nature informationsdi...,springer berlin heidelberg pp der nature avail...,society american et available al internet engi...,reviews online et choice al der pp american vo...,anon research nd news et al doi medical editor...
ieee conference international th journal systems proceedings system wang based,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,doi review society american reviews internet n...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,anon research nd news et al doi medical editor...,elsevier society pp bv vol engineers mechanica...,cheminform abstract nono nature informationsdi...,journal available pp online doi american inter...,society american et available al internet engi...
de ieee conference international press springer la university pp th,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,american elsevier medical association bv revie...,springer berlin heidelberg pp der nature avail...,journal available pp online doi american inter...,elsevier society pp bv vol engineers mechanica...,cheminform abstract nono nature informationsdi...,society american et available al internet engi...,doi review society american reviews internet n...,anon research nd news et al doi medical editor...
press university oxford pp cambridge ieee online springer conference anon,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,springer berlin heidelberg pp der nature avail...,doi review society american reviews internet n...,american elsevier medical association bv revie...,elsevier society pp bv vol engineers mechanica...,anon research nd news et al doi medical editor...,et al elsevier bv wang research internet zhang...,journal available pp online doi american inter...,cheminform abstract nono nature informationsdi...
springer berlin heidelberg pp der nature available und online die,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,doi review society american reviews internet n...,elsevier society pp bv vol engineers mechanica...,anon research nd news et al doi medical editor...,cheminform abstract nono nature informationsdi...,journal available pp online doi american inter...,society american et available al internet engi...
available online anon internet reviews choice ieee book supsup ppp,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,doi review society american reviews internet n...,elsevier society pp bv vol engineers mechanica...,anon research nd news et al doi medical editor...,cheminform abstract nono nature informationsdi...,journal available pp online doi american inter...,et al elsevier bv wang research internet zhang...
et al elsevier bv wang research internet zhang li cancer,available online anon internet reviews choice ...,der und die für des internet von zeitschrift v...,doi review society american reviews internet n...,springer berlin heidelberg pp der nature avail...,et al elsevier bv wang research internet zhang...,american elsevier medical association bv revie...,anon research nd news et al doi medical editor...,cheminform abstract nono nature informationsdi...,journal available pp online doi american inter...,elsevier society pp 